# Recognize already existing persons in Geovistory from the BHP

In [ ]:
import pandas as pd

import toolkit as tk

from splink.duckdb.duckdb_linker import DuckDBLinker
from splink.charts import waterfall_chart
import splink.duckdb.duckdb_comparison_library as cl

## Load data

Here we are going to load Geovistory's data on one side, and BHP's data on the other, the goal being of identifing exiting persons of the BHP into Geovistory.

In [ ]:
persons_geov = pd.read_csv('../../data/persons-geov.csv', sep=';')
tk.set_types(persons_geov, {'name':'string', 'gender':'string', 'birth_year':'int', 'death_year':'int'})
persons_geov.reset_index(inplace=True)

print('Geovistory\'s persons:')
tk.infos(persons_geov)

In [ ]:
persons_bhp = pd.read_csv('../../data/persons-bhp.csv', sep=';')
tk.set_types(persons_bhp, {'name':'string', 'gender':'string', 'birth_year':'int', 'death_year':'int'})
persons_bhp.reset_index(inplace=True)

print('BHP\'s persons:')
tk.infos(persons_bhp)

## Settings

### Prediction generation blocking rules

Here are the set blocking rules to generate the pairwise comparisons; in other words all record couples that do not statisfied at least one of those rules will not be considered in final predictions. 

This is so to avoid to have to do all comparisons which would be heavy (would take forever) for a computer to do. In our case it would be length of BHP data times length of Geovistory data times number of column, so 62k times 147k times 4 = 37G comparisons.


In [ ]:
blocking_rules_predictions = [
    "l.birth_year = r.birth_year and levenshtein(l.name, r.name) <= 3",
    "l.death_year = r.death_year and levenshtein(l.name, r.name) <= 3",
]

for br in blocking_rules_predictions:
    print(f"Blocking rule: <{br}>")

### Final comparisons

We also set multiple comparison rules, which describe how the comparison will be executed on 2 records.

In [ ]:
comparisons = [
    # cl.levenshtein_at_thresholds("name", 1), # Because we want matches with only a typo be more close to a matching than is it is another spelling
    cl.levenshtein_at_thresholds("name", 3), # If a name has another spelling (like phonetics)
    cl.exact_match("gender"), # Because we have controlled vocabulary for the gender
    cl.exact_match("birth_year"), # This takes the assumption that there is no typo possible on birth year
    cl.exact_match("death_year"), # This takes the assumption that there is no typo possible on death year
]

for br in comparisons:
    print(f"{br}")

In [ ]:
settings = {
    "link_type": "link_only", # Describe the fact that we want to merge 2 dataframe and that one may already have some of the second one.
    "unique_id_column_name": "index", # Each dataframe has to have a unique key for each line, here we tell Splink, what is the name of the column
    "blocking_rules_to_generate_predictions": blocking_rules_predictions,
    "comparisons": comparisons,
    "retain_matching_columns": True, # To have waterfall charts
    "retain_intermediate_calculation_columns": True, # To have waterfall charts
}

## The Model

### Creating the model

The next chart displays how much final comparisons the model will have to predict. Basically this total number will be an upper boundary of the final prediction table length.

In [ ]:
linker = DuckDBLinker(
    [persons_geov, persons_bhp], 
    settings, 
    input_table_aliases=["geov", "bhp"] # To have custom names in comparison table
)

linker.cumulative_num_comparisons_from_blocking_rules_chart()

### Training blocking rule

In order to train our model, we need to estimate the $m$ and $u$ parameters of the Fellegi-Sunter model associated with and EM algorithm (more of that [here](https://www.robinlinacre.com/maths_of_fellegi_sunter/) and [here](https://www.robinlinacre.com/em_intuition/)).

The $u$ parameter will be estimated using random sampling. This is valid (as explained [here](https://moj-analytical-services.github.io/splink/linker.html#splink.linker.Linker.estimate_u_using_random_sampling)) because there is a very low probability for 2 randomly picked records to be the same person. We just have to make sure that the sample taken is large enough to correctly train this parameter.

For the $m$ parameter, to have a powerfull model, we can not take such an hypothesis; we need to train the model (statistically) on the data. As before, since we can not take the full data, we need to filter out comparisons so that it is manageable in a reasonnable time. See the next display to see rules we took. 

How to interpret those rules? Basically, for all column (present in the comparison rules above) not being in a training blocking rules, we will estimate the $m$ parameter on pairwise comparisons validated by the rule. In other word, if the `gender` column does not appear in the rule, it means that the $m$ parameter for the column `gender` will be trained (calculated) on all the pairwaise comparisons remaining after beeing filter by the rule.

This also implies that all columns MUST not appear in at least one rule, otherwise we can never train the $m$ parameter.

The training blocking rules:

In [ ]:
blocking_rules_training = [
    "l.birth_year = r.birth_year and levenshtein(l.name, r.name) <= 3",
    "l.death_year = r.death_year and levenshtein(l.name, r.name) <= 3",
    "l.birth_year = r.birth_year and l.death_year = r.death_year",
]

for br in blocking_rules_training:
    print(f"Blocking rule: <{br}>")

### Training phase

In [ ]:
z = linker.estimate_u_using_random_sampling(target_rows=2e7)

for br in blocking_rules_training:
    z = linker.estimate_parameters_using_expectation_maximisation(br)

## Predictions

### What has been learned?

Now that we have our model trained, first, lets look at what did the model learn about our data:

In [ ]:
linker.match_weights_chart()

Here we can observe how a response level (on the left) influences the matching probability.

More particularly we observe that the gender has a very low influence on the result, just on the contrary of the name.

### Persons identified

The next table is an extract of the 50 most probable matchings.
Each line represents a pairwise comparison. We can see the probability (computed by the trained model), and each column put aside another in order to be more human readable. A copy is available as a CSV table in the `data` folder.

In [ ]:
# results = linker.predict(threshold_match_probability=0.9)
results = linker.predict()
results_df = results.as_pandas_dataframe().sort_values(by='match_probability', ascending=False)
print('Result number:', len(results_df))

readable = results_df[['match_probability', 'source_dataset_l', 'source_dataset_r', 'index_l', 'index_r', 'name_l', 'name_r', 'gender_l', 'gender_r', 'birth_year_l', 'birth_year_r', 'death_year_l', 'death_year_r']].copy()
readable.rename(columns={
    'match_probability':'proba', 
    'index_l': 'id_bhp', 
    'index_r': 'pk_geov',
    'name_l': 'bhp_name',
    'name_r': 'geov_name',
    'gender_l': 'bhp_gender',
    'gender_r': 'geov_gender',
    'birth_year_l': 'bhp_birth_year',
    'birth_year_r': 'geov_birth_year',
    'death_year_l': 'bhp_death_year',
    'death_year_r': 'geov_death_year'
}, inplace=True)
tk.set_types(readable, {
    'bhp_birth_year': 'int',
    'geov_birth_year': 'int',
    'bhp_death_year': 'int',
    'geov_death_year': 'int',
})
readable['proba'] = [tk.percent(p) for p in readable['proba']]
readable.drop(columns=['source_dataset_l', 'source_dataset_r'], inplace=True)

readable.to_csv('../../data/bhp_geov_entity_recognition.csv', sep=";", index=False, quoting=2)
readable.head(50)

### Details

The next chart gives us details about a particular pairwise comparison. The example can be changed with the bottom slider. More information are available on mouse over the different elements.

This chart helps us understand why the model answered the provided response.

For conveniance, only the first 1000 comparisons are available through this chart.

In [ ]:
records_to_plot = results_df.head(1000).to_dict(orient="records")
linker.waterfall_chart(records_to_plot, filter_nulls=False)